In [1]:
import numpy as np
import torch
from lib.models.models import UniformRate
import ml_collections
from config.config_train_sample import get_config
from lib.utils import utils


In [ ]:
"""
Now, when we minimize LCT, we are sampling (x, x ̃) from the forward process and then maximizing the assigned model probability for 

the pairing in the reverse direction, just as in LDT. The slight extra complexity comes from the fact we areconsidering the case 

when xk = xk+1 and the case when xk ̸= xk+1 separately. When xk = xk+1, this corresponds to the first term in LCT which we can see 

is minimizing the reverse rate out of x which is exactly maximizing the model probability for no transition to occur. When xk ̸= xk+1, 

this corresponds to the second term in LCT, which is maximizing the reverse rate from x ̃ to x which in turn maximizes the model probability 

for the x ̃ to x transition to occur.
"""

In [2]:
# Fragen Yannick:
# Wieso xt samplen from q_t|0 and then one transition to x_tilde => and then logits = net(x_tile, t)
#    # q_{t|0} (x ̃|x_0) =>  qt0_numer_reg = qt0.view(B, S, S)

# 2 forward passed: p^{θ}_{0|t}(x0|x) to calculate Rˆ{θ}_t(x, x′) and p^{θ}_{0|t}(x0|x ̃) to calculate Rˆ{θ}_t(x ̃, x). This is wasteful
rate_const = 1
S = 3
B = 2
D = 4
device = 'cpu'
rate = rate_const * np.ones((S, S))
rate = rate - np.diag(np.diag(rate))
#print(rate)
rate = rate - np.diag(np.sum(rate, axis=1))

#print(rate)
rate_matrix = torch.from_numpy(rate).float()

rate_matrix = torch.tile(rate_matrix.view(1, S, S), (B, 1, 1))
#print(rate_matrix.shape)
v = torch.tensor([1, 2, 3])
matrix = torch.diag_embed(v)
#print(matrix)

In [8]:
#-----------Transition matrix q_t|0: x0 -> xt ---------------------
cfg = get_config()
cfg.data.S = S
cfg.model.rate_const = rate_const
uni = UniformRate(cfg, 'cpu')
ts = torch.rand((B,))
qt0 = uni.transition(ts)
x0= torch.randint(low=0, high=S, size=(B, D), dtype=torch.int)
#print(x0)
#print(qt0, qt0.shape)
qt0_rows_reg = qt0[
    torch.arange(B, device=device).repeat_interleave(
        D
    ),  # repeats every element 0 to B-1 D-times
    x0.flatten().long(),  # minibatch.flatten() => (B, D) => (B*D) (1D-Tensor)
    :,
]
print(qt0_rows_reg, qt0_rows_reg.shape)
b = utils.expand_dims(torch.arange(B), (tuple(range(1, x0.dim()))))
qt0_rows_reg2 = qt0[b, x0].view(-1, S)
print(qt0_rows_reg2 == qt0_rows_reg)
print(qt0_rows_reg2, qt0_rows_reg2.shape)

x_t_cat = torch.distributions.categorical.Categorical(qt0_rows_reg)
x_t = x_t_cat.sample().view(B, D)
print(x_t, x_t.shape)

tensor([[0.2817, 0.2817, 0.4367],
        [0.4367, 0.2817, 0.2817],
        [0.2817, 0.4367, 0.2817],
        [0.4367, 0.2817, 0.2817],
        [0.7028, 0.1486, 0.1486],
        [0.1486, 0.7028, 0.1486],
        [0.1486, 0.7028, 0.1486],
        [0.1486, 0.1486, 0.7028]]) torch.Size([8, 3])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[0.2817, 0.2817, 0.4367],
        [0.4367, 0.2817, 0.2817],
        [0.2817, 0.4367, 0.2817],
        [0.4367, 0.2817, 0.2817],
        [0.7028, 0.1486, 0.1486],
        [0.1486, 0.7028, 0.1486],
        [0.1486, 0.7028, 0.1486],
        [0.1486, 0.1486, 0.7028]]) torch.Size([8, 3])
tensor([[0, 1, 2, 2],
        [0, 1, 0, 0]]) torch.Size([2, 4])


In [135]:
#-------------- Transition rate: x_t -> x_tilde ------------------
rate = uni.rate(ts)
#print(rate, rate.shape) # B, S, S
rate_vals_square = rate[
        torch.arange(B, device=device).repeat_interleave(D), x_t.long().flatten(), :
    ]
#print(rate_vals_square, rate_vals_square.shape)
rate_vals_square[
        torch.arange(B * D, device=device), x_t.long().flatten()
    ] = 0.0 
print(rate_vals_square, rate_vals_square.shape)

rate_vals_square = rate_vals_square.view(B, D, S)
print(rate_vals_square, rate_vals_square.shape)

rate_vals_square_dimsum = torch.sum(rate_vals_square, dim=2).view(B, D)
print(rate_vals_square_dimsum, rate_vals_square_dimsum.shape)

square_dimcat = torch.distributions.categorical.Categorical(rate_vals_square_dimsum)

square_dims = square_dimcat.sample() # sampled where transition takes places in every row of B
print("Where transition", square_dims, square_dims.shape)

rate_new_val_probs = rate_vals_square[
    torch.arange(B, device=device), square_dims, :
]  # (B, S)
print(rate_new_val_probs, rate_new_val_probs.shape)

square_newvalcat = torch.distributions.categorical.Categorical(
    rate_new_val_probs
)

# Shape: (B,) mit Werten im Bereich [0, S)
square_newval_samples = (
    square_newvalcat.sample()
)
print("Transition value", square_newval_samples, square_newval_samples.shape)

x_tilde = x_t.clone()
        # noisy image 
x_tilde[torch.arange(B, device=device), square_dims] = square_newval_samples
print(x_t)
print(x_tilde)


tensor([[1., 1., 0.],
        [0., 1., 1.],
        [0., 1., 1.],
        [1., 1., 0.],
        [0., 1., 1.],
        [1., 1., 0.],
        [0., 1., 1.],
        [1., 0., 1.]]) torch.Size([8, 3])
tensor([[[1., 1., 0.],
         [0., 1., 1.],
         [0., 1., 1.],
         [1., 1., 0.]],

        [[0., 1., 1.],
         [1., 1., 0.],
         [0., 1., 1.],
         [1., 0., 1.]]]) torch.Size([2, 4, 3])
tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.]]) torch.Size([2, 4])
Where transition tensor([3, 0]) torch.Size([2])
tensor([[1., 1., 0.],
        [0., 1., 1.]]) torch.Size([2, 3])
Transition value tensor([0, 2]) torch.Size([2])
tensor([[2, 0, 0, 2],
        [0, 2, 0, 1]])
tensor([[2, 0, 0, 0],
        [2, 2, 0, 1]])


In [136]:
#-----------ELBO-------------------
mask_reg = torch.ones((B, D, S), device=device)

mask_reg[
    torch.arange(B, device=device).repeat_interleave(D),
    torch.arange(D, device=device).repeat(B),
    x_tilde.long().flatten(),
] = 0.0
print(x_tilde)
print(mask_reg, mask_reg.shape)

tensor([[2, 0, 0, 0],
        [2, 2, 0, 1]])
tensor([[[1., 1., 0.],
         [0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.]],

        [[1., 1., 0.],
         [1., 1., 0.],
         [0., 1., 1.],
         [1., 0., 1.]]]) torch.Size([2, 4, 3])


In [148]:
qt0_numer_reg = qt0.view(B, S, S)
print(qt0_numer_reg , qt0_numer_reg.shape)
# q_{t|0} (x|x_0)
qt0_denom_reg = (
    qt0[
        torch.arange(B, device=device).repeat_interleave(D),
        :,
        x_tilde.long().flatten(),
    ].view(B, D, S)
    + 1e-6
)
#print(qt0_denom_reg, qt0_denom_reg.shape)

#print(rate, rate.shape)
rate_vals_reg = rate[
    torch.arange(B, device=device).repeat_interleave(D),
    :,
    x_tilde.long().flatten(),
].view(B, D, S)
print(rate_vals_reg, rate_vals_reg.shape)
print((mask_reg * rate_vals_reg))
reg_tmp = (mask_reg * rate_vals_reg) @ qt0_numer_reg.transpose(1, 2)
print(reg_tmp, reg_tmp.shape)

tensor([[[0.3818, 0.3091, 0.3091],
         [0.3091, 0.3818, 0.3091],
         [0.3091, 0.3091, 0.3818]],

        [[0.3917, 0.3042, 0.3042],
         [0.3042, 0.3917, 0.3042],
         [0.3042, 0.3042, 0.3917]]]) torch.Size([2, 3, 3])
tensor([[[ 1.,  1., -2.],
         [-2.,  1.,  1.],
         [-2.,  1.,  1.],
         [-2.,  1.,  1.]],

        [[ 1.,  1., -2.],
         [ 1.,  1., -2.],
         [-2.,  1.,  1.],
         [ 1., -2.,  1.]]]) torch.Size([2, 4, 3])
tensor([[[1., 1., -0.],
         [-0., 1., 1.],
         [-0., 1., 1.],
         [-0., 1., 1.]],

        [[1., 1., -0.],
         [1., 1., -0.],
         [-0., 1., 1.],
         [1., -0., 1.]]])
tensor([[[0.6909, 0.6909, 0.6182],
         [0.6182, 0.6909, 0.6909],
         [0.6182, 0.6909, 0.6909],
         [0.6182, 0.6909, 0.6909]],

        [[0.6958, 0.6958, 0.6083],
         [0.6958, 0.6958, 0.6083],
         [0.6083, 0.6958, 0.6958],
         [0.6958, 0.6083, 0.6958]]]) torch.Size([2, 4, 3])
